# Running age model on many stars and comparing with Jason's ages.

In [26]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from isochrones.mist import MIST_Isochrone
from isochrones import StarModel
mist = MIST_Isochrone()
import emcee
import priors
import corner
import h5py

Load the data

In [2]:
df = pd.read_csv("kane_cks_tdmra_dr2.csv")

Estimate B-V colours for stars based on their G_BP - G_RP colours. This is a crude estimated used only if a gyro-only lhf is used.

In [3]:
def bv_2_bprp(bv):
    """
    Numbers from https://arxiv.org/pdf/1008.0815.pdf
    """
    a, b, c, d = .0981, 1.4290, -.0269, .0061  # sigma = .43
    return a + b*bv + c*bv**2 + d*bv**3

def bprp_2_bv(bprp):
    """
    Try to find the analytic version of this, please!
    """
    bv_iter = np.linspace(0., 2., 10000)
    bprp_pred = [bv_2_bprp(bv) for bv in bv_iter]
    diffs = bprp - np.array(bprp_pred)
    # plt.plot(bprp_pred, diffs**2)
    return bv_iter[np.argmin(diffs**2)]

Define the gyrochronology model. I'm using the Barnes (2003) functional form with my (2015) parameters.

In [4]:
def gyro_model(ln_age, bv):
    """
    Given a B-V colour and an age, predict a rotation period.
    Returns log(age) in Myr.
    parameters:
    ----------
    logage: (array)
        The log age of a star: log10(age) in years.
    bv: (array)
        The B-V colour of a star.
    """
    age_myr = (10**ln_age)*1e-6
    
    a, b, c, n = [.4, .31, .45, .55]
    
    # return a*(age_myr)**n * (bv - c)**b
    log_P = n*np.log10(age_myr) + np.log10(a) + b*np.log10(bv-c)
    return 10**log_P

Define the log prior and the log posterior functions.

In [5]:
def lnprior(params):
    """
    lnprior on all parameters.
    params need to be linear except age which is log10(age [yr]).
    """
    
    # log Priors over age, metallicity and distance. (The priors in priors.py are not in log)
    age_prior = np.log(priors.age_prior(params[1]))
    feh_prior = np.log(priors.feh_prior(params[2]))
    distance_prior = np.log(priors.distance_prior(np.exp(params[3])))

    # Uniform prior on extinction.
    mAv = (0 <= params[4]) * (params[4] < 1)  # Prior on A_v
    mAv = mAv == 1
    
    # Uniform prior on mass
    m = (-20 < params[0]) * (params[0]) < 20  # Broad bounds on mass.
            
    if mAv and m and np.isfinite(age_prior) and np.isfinite(distance_prior):
        return age_prior + feh_prior + distance_prior
    
    else:
        return -np.inf

    
def lnprob(lnparams, *args):
    # Transform mass and distance back to linear.
    params = lnparams*1
    params[0] = np.exp(lnparams[0])
    params[3] = np.exp(lnparams[3])
    
    mod, period, period_err, bv_est, gyro_only, iso_only = args
    
    B = mist.mag["B"](*params)
    V = mist.mag["V"](*params)
    bv = B-V
    
    # If the prior is -inf, don't even try to calculate the isochronal likelihood.
    lnpr = lnprior(params)
    if lnpr == -np.inf:
        return lnpr
    
    else:
    
        if iso_only:
            return mod.lnlike(params) + lnpr
        
        else:
            if bv > .45:
                gyro_lnlike = -.5*((period - gyro_model(params[1], bv)) /period_err)**2
            else:
                gyro_lnlike = 0
    
        # B-V is estimated from mass, etc, so you need to use a different B-V estimate if gyro_only.
        if gyro_only:
            return -.5*((period - gyro_model(params[1], bv_est)) /period_err)**2 + lnpr
    
        else:
            return mod.lnlike(params) + gyro_lnlike + lnpr

Define the function that infers the age of a single star via MCMC.

In [6]:
def inits(df, star_ind):
    # Set the initial values
    mass_init = df.mass.values[star_ind]
    age_init = 2e9
    feh_init = df.feh.values[star_ind]
    distance_init = 1./df.parallax.values[star_ind]
    Av_init = .1

    # sample in ln(mass), log10(age) and ln(distance).
    p_init = np.array([np.log(mass_init), np.log10(age_init), feh_init, np.log(distance_init), Av_init])

    # Set up the StarModel object needed to calculate the likelihood. 
    param_dict = {"J": (df.J.values[star_ind], df.J_err.values[star_ind]),
                  "H": (df.H.values[star_ind], df.H_err.values[star_ind]),
                  "K": (df.K.values[star_ind], df.K_err.values[star_ind]),
                  "Kepler": (df.kepmag.values[star_ind], df.kepmag_err.values[star_ind])
                  "teff": (df.teff.values[star_ind], df.teff_err1.values[star_ind]),
                  "logg": (df.logg.values[star_ind], df.logg_err1.values[star_ind]),
                  "feh": (df.feh.values[star_ind], df.feh_err1.values[star_ind]),
                  "parallax": (df.parallax.values[star_ind],
                  (df.parallax_error.values[star_ind]))}  # Isochrones.py takes milliarcseconds
    mod = StarModel(mist, **param_dict)
    return param_dict, p_init, mod


def run_mcmc(df, param_dict, p_init, mod, star_ind, nwalkers=24):

    ndim = 5
    args = [mod, df.period.values[star_ind], 
            .5*(df.period_errm.values[star_ind] + df.period_errp.values[star_ind]),
            df.BV_color.values[star_ind], False, False]

    p0 = [p_init + np.random.randn(ndim)*1e-4 for i in range(nwalkers)]

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=args)
    p0, _, _ = sampler.run_mcmc(p0, 5000);
    sampler.reset()
    sampler.run_mcmc(p0, 10000);
    
    samples = sampler.flatchain
    samples[:, 0] = np.exp(sampler.flatchain[:, 0])
    samples[:, 3] = np.exp(sampler.flatchain[:, 3])
    return samples

Run mcmc on multiple stars and save the samples.

In [27]:
RESULTS_DIR = "samples"

# Create estimated colour column.
bv_color = []
for i in range(np.shape(df)[0]):
    bv_color.append(bprp_2_bv(df.phot_bp_mean_mag.values[i] - df.phot_rp_mean_mag.values[i]))

# Create df
cks_df = pd.DataFrame(dict({"source_id": df.source_id,
                            "J": df.jmag.values, 
                            "J_err": df.jmag.values*.05,
                            "H": df.hmag.values,
                            "H_err": df.hmag.values*.05,
                            "K": df.kmag.values,
                            "K_err": df.kmag.values*.05,
                            "kepmag": df.kepmag.values,
                            "kepmag_err": df.kepmag.values*.05,
                            "mass": df.iso_smass.values, 
                            "feh": df.cks_smet.values,
                            "feh_err1": df.cks_smet_err1.values,
                            "parallax": df.parallax.values,
                            "parallax_err": df.parallax_error.values,
                            "teff": df.cks_steff.values,
                            "teff_err1": df.cks_steff_err1.values,
                            "logg": df.cks_slogg.values,
                            "logg_err1": df.cks_slogg_err1.values,
                            "BV_color": np.array(bv_color)}))

# Load the Sanders catalogue and merge with the CKS one.
file = "gaia_spectro.hdf5"
data = Table.read(file)
sanders = data.to_pandas()
sdf = pd.merge(cks_df, sanders, on="source_id")

nstars = 1
ages, age_errp, age_errm = [np.zeros(nstars) for i in range(3)]

def ensemble(nstars):
    
    for star in range(nstars):
        
        param_dict, p_init, mod = inits(sdf, star)

        samples = run_mcmc(df, param_dict, p_init, mod, star)
        ages[star] = np.median(samples[:, 1])
        
        f = h5py.File("samples/{0}.h5".format(star), "w")
        data = f.create_dataset("samples", np.shape(samples))
        data[:, :] = samples
        f.close()
        
ensemble(nstars)

(546,) (546,)


NameError: name 'inits' is not defined

Now load Sanders sample and compare with their ages.

In [10]:
#corner.corner(samples, labels=["mass [M_sun]", "log10(age [yr])", "[Fe/H]", "distance [pc]", "A_v"])

In [19]:
file = "gaia_spectro.hdf5"
from astropy.table import Table
data = Table.read(file)
print(data.meta["COMMENT"])
sanders = data.to_pandas()

['Distances, ages, masses and extinctions for spectroscopic surveys combined with Gaia DR2', 'Isochrone set(s) used = PARSEC version 1.2S eta=0.2 http://stev.oapd.inaf.it/cgi-bin/cmd', 'Adopted prior: 2018 prior described in Sanders et al. (2018)', 'specobjid is the unique identifier for the SEGUE survey', 'CNAME is the unique identifier for the GES survey', 'sobject_id is the unique identifier for the GALAH survey', 'APOGEE_ID is the unique identifier for the APOGEE survey', 'raveid is the unique identifier for the RAVE Cannon survey', 'raveid is the unique identifier for the RAVE DR5 survey', 'obsid is the unique identifier for the LAMOST survey', 'source_id is the Gaia DR2 cross-matched source', 'dm and dm_err give the distance modulus with associated error', 'dist and dist_err give the distance (kpc) with associated error', 'par and par_err give the parallax (mas) output from the code (not directly from Gaia) with associated error', 'mass and mass_err give the mass (Msun) with asso

In [20]:
sdf = pd.merge(sanders, df, on="source_id")
print(np.shape(sanders), np.shape(df), np.shape(sdf))
sanders.keys()
#print(sanders.Z)

(4906746, 62) (546, 246) (326, 307)


Index(['obsid', 'source_id', 'dm', 'dm_err', 'dist', 'dist_err', 'par',
       'par_err', 'log10_age', 'log10_age_err', 'mass', 'mass_err', 'Z',
       'Z_err', 'log10_av', 'log10_av_err', 'log10_teff', 'log10_teff_err',
       'logg', 'logg_err', 'dm_log10age_corr', 'log10age_Z_corr', 'dm_Z_corr',
       'l', 'b', 's', 'vlos', 'mu_l', 'mu_b', 'R', 'phi', 'z', 'vR', 'vphi',
       'vz', 'JR', 'Lz', 'Jz', 'Rc', 's_err', 'vlos_err', 'mu_l_err',
       'mu_b_err', 'R_err', 'phi_err', 'z_err', 'vR_err', 'vphi_err', 'vz_err',
       'JR_err', 'Lz_err', 'Jz_err', 'Rc_err', 'flag', 'survey', 'raveid',
       'APOGEE_ID', 'sobject_id', 'CNAME', 'specobjid', 'duplicated', 'best'],
      dtype='object')

In [ ]:
# Create estimated colour column.
bv_color = []
for i in range(np.shape(sdf)[0]):
    bv_color.append(bprp_2_bv(sdf.phot_bp_mean_mag.values[i] - sdf.phot_rp_mean_mag.values[i]))

# Create df
mydf = pd.DataFrame(dict({"mass": sdf.mass.values, 
                          "feh": sdf.cks_smet.values,
                          "feh_err1": df.cks_smet_err1.values,
                          "parallax": df.parallax.values,
                          "parallax_err": df.parallax_error.values,
                          "teff": 10**df.log10_teff.values,
                          "teff_err1": abs(10**df.log10_teff.values 
                                             - 10**(df.log10_teff.values + df.log10_teff_err.values),
                          "logg": df.logg.values,
                          "logg_err1": df.logg_err.values
                          "BV_color": np.array(bv_color)}))


mass_init = sanders.mass.values[star_ind]
age_init = 2e9
feh_init = san.cks_smet.values[star_ind]
distance_init = 1./df.parallax.values[star_ind]
Av_init = .1

# sample in ln(mass), log10(age) and ln(distance).
p_init = np.array([np.log(mass_init), np.log10(age_init), feh_init, np.log(distance_init), Av_init])

# Set up the StarModel object needed to calculate the likelihood. 
param_dict = {"J": (df.jmag.values[star_ind], df.jmag.values[star_ind]*.05),
            "H": (df.hmag.values[star_ind], df.hmag.values[star_ind]*.05),
            "K": (df.kmag.values[star_ind], df.kmag.values[star_ind]*.05),
            "teff": (df.cks_steff.values[star_ind], df.cks_steff_err1.values[star_ind]),
            "logg": (df.cks_slogg.values[star_ind], df.cks_slogg_err1.values[star_ind]),
            "feh": (df.cks_smet.values[star_ind], df.cks_smet_err1.values[star_ind]),
            "parallax": (df.parallax.values[star_ind],
            (df.parallax_error.values[star_ind]))}  # Isochrones.py takes milliarcseconds
mod = StarModel(mist, **param_dict)

# Create estimated colour column.
bv_color = []
for i in range(np.shape(df)[0]):
    bv_color.append(bprp_2_bv(df.phot_bp_mean_mag.values[i] - df.phot_rp_mean_mag.values[i]))
df["BV_color"] = np.array(bv_color)